# Loading dependencies

In [1]:
from models.gutenberg_loader import GutenbergBooksLoader
from gutenberg_rag import get_embedding_model

import os
# root folder
dir_path = os.path.dirname(os.path.realpath('__file__'))


# Loading books

In [2]:
book = GutenbergBooksLoader.load_unstructured_from_filesystem(f"{dir_path}/dataset/pg19942.txt")


In [3]:
book[68:100]

[Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'filetype': 'text/plain', 'category': 'Title'}, page_content="[Illustration: VOLTAIRE'S CANDIDE]"),
 Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'filetype': 'text/plain', 'category': 'Title'}, page_content='CANDIDE'),
 Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 

In [4]:
len(book)

935

# Chunking

In [5]:
from controller.gutenberg_text_splitter import split_recursively_books_to_chunks

book_chunks = split_recursively_books_to_chunks(book)
book_chunks[100:120]

[Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'parent_id': '4af12309cfc426b688ccf4bbca3a3430', 'filetype': 'text/plain', 'category': 'NarrativeText'}, page_content='"Yes, sir, that is my height," answered he, making a low bow.'),
 Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'parent_id': '4af12309cfc426b688ccf4bbca3a3430', 'filetype': 'text/plain', 'category': 'NarrativeText'}, page_content='"Come, sir, seat yourself; not only will we pay your reckoning, but we will never suffer such a man as you to want money; men are o

In [6]:
len(book_chunks)

1081

# Embedding

In [7]:

embedding = get_embedding_model()
#print(embedding)

/home/dria/miniconda3/envs/booksummary-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/dria/miniconda3/envs/booksummary-env/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# VectorStore


In [8]:
from gutenberg_rag import create_faiss_vector_store, load_faiss_vector_store

In [9]:

#vector_store = create_faiss_vector_store(book, embedding)

### Cache the result


### Load from cache 

In [10]:

# vector_store = FAISS.load_local(
#     "faiss_vector_store", embedding, allow_dangerous_deserialization=True) 
    


In [13]:
vector_store  =  load_faiss_vector_store("faiss_vector_store", embedding)

# Retrieve information

In [14]:
#### Retriever 
retriever = vector_store.as_retriever(search_kwargs={ "k": 5 , 'score_threshold': 0.9})
relevant_docs = retriever.get_relevant_documents("Why CANDIDE WAS EXPELLED from CASTLE.")
relevant_docs

/home/dria/miniconda3/envs/booksummary-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'parent_id': '69f68528005702a734891d9e9db71148', 'filetype': 'text/plain', 'category': 'NarrativeText'}, page_content='I. How Candide was brought up in a Magnificent Castle, and how he was expelled thence                                          1'),
 Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'parent_id': '5e402b4d964077a0e996d0163fa69099', 'filetype': 'text/plain', 'category': 'UncategorizedText'}, page_content='HOW CANDIDE WAS BROUGHT UP IN A MAGNIFICENT CASTLE, AND HOW 

### Parent Loader

In [23]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
                                                   chunk_size=512,
                                                   chunk_overlap=20, 
                                                   length_function=len,
                                                   #separators= ["\n\n", "***\n\n\n\n\n", "\n\n\n\n\" ],
                                                   is_separator_regex=False)

In [43]:
store = InMemoryStore()
long_context_retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=store,
    child_splitter=text_splitter,
    id_key="parent_id"
)

In [44]:
long_context_retriever.add_documents(book_chunks)

In [31]:
list(store.yield_keys())

['385c1f56-4086-40cf-b1d1-5b6e126e10d3',
 '10f8564b-0943-44c8-84fa-5f98caa3ac8a',
 'bb2e1201-366c-4b72-aaec-67f92e83df43',
 '847e353b-75b6-4b9e-b742-3c75a8b2cef6',
 '9bfee753-9821-4e9d-950b-dd760765baf0',
 'b4971933-b0d6-4794-96fc-0026774425f6',
 '0635c30c-3558-4fd8-ab62-0e9b4e642fbc',
 '82329ece-6387-4410-a144-e9b504d1e817',
 '1fec7a35-e484-43c9-b1c0-ee6fe0af84f8',
 'faabad53-4c74-4cbc-8692-d8cedd3ac912',
 '55f236b1-956e-4b2f-a7d3-4a9166728eca',
 '1cd0e9b5-1a74-464c-9a5e-a4b9920be752',
 'c527c32b-40e6-470b-b83c-41559b71eb65',
 '405eedac-1117-41b4-bb90-9bd388847b41',
 '170b6617-3674-4176-8078-a2b4095a2899',
 '3ff794a3-bc6b-4ee1-bd75-1945fe3f1467',
 'ff1e2c23-73d7-47c0-a12d-40a304ab0a78',
 '43343b55-def4-4f61-b550-b0a40c47fb9c',
 '017ddbaa-97c8-4ef3-90a4-891e3267f6f0',
 '60016a0e-03dc-4e2a-a675-72d304fe50e6',
 '4f5dbd77-863d-44b2-8f0a-f15e35add07c',
 '944b7b18-013c-46e0-9d5c-3245df9202d4',
 'bb7c2cfd-b608-48da-b9d1-03c86885e9ad',
 '65de1c2a-968b-4c77-8662-9f69f649ea59',
 'cd407fb9-f4de-

In [41]:
for index, item in enumerate(store.yield_keys()):
    print(item)
    print(store.mget([item]))
    print(index)

385c1f56-4086-40cf-b1d1-5b6e126e10d3
[Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'filetype': 'text/plain', 'category': 'Title'}, page_content='The Project Gutenberg eBook of Candide')]
0
10f8564b-0943-44c8-84fa-5f98caa3ac8a
[Document(metadata={'source': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset/pg19942.txt', 'file_directory': '/home/dria/Documents/Projets/BookSummary/llm-assessment/dataset', 'filename': 'pg19942.txt', 'last_modified': '2024-07-20T06:55:10', 'languages': ['eng'], 'parent_id': '7ed5f5bbfd2a21ad8eea631f4ef22ce9', 'filetype': 'text/plain', 'category': 'NarrativeText'}, page_content='This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restr

# LLM

In [16]:
from gutenberg_rag import get_llm

llm = get_llm()

# Question-Answer

In [18]:
from langchain.chains import RetrievalQA

qa  =   RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

In [20]:
qa({"query" : "what happened to candide in the first chapter?"})

{'query': 'what happened to candide in the first chapter?',
 'result': 'The context is from the first chapter of "Candide" by Voltaire, where Candide is mentioned as having been killed by Martine. The passage does not provide any information about his death or how it happened. It\'s best to consult the original text for more details.'}

In [21]:
qa({"query" : "why candide was expelled from the castle?"})

{'query': 'why candide was expelled from the castle?',
 'result': 'The context provided does not provide a clear answer to the question at hand. The passage only mentions that Candide was "brought up" in a Magnifiqueant Castle and then "expelled" thereafter, but it doesn\'t provide any further details about why he was expelled or what happened afterward.'}

# Generate a summary

In [ ]:
from controller.summary_generator import SummaryGeneratorController

summary = SummaryGeneratorController.generate_summary_from_long_gutenberg_book(book, llm)

In [ ]:
#summary

# Evaluation


In [6]:
from langchain_community.document_loaders import DirectoryLoader
from controller.summary_generator import SummaryGeneratorController
from gutenberg_rag import get_llm



In [7]:
llm = get_llm()

In [2]:
loader = DirectoryLoader("./dataset/pg19942/pg19942-chapters")

In [3]:
book = loader.load()

In [4]:
book

[Document(metadata={'source': 'dataset/pg19942/pg19942-chapters/04.txt'}, page_content='HOW CANDIDE MADE HIS ESCAPE FROM THE BULGARIANS, AND WHAT AFTERWARDS BECAME OF HIM.\n\nThere was never anything so gallant, so spruce, so brilliant, and so well disposed as the two armies. Trumpets, fifes, hautboys, drums, and cannon made music such as Hell itself had never heard. The cannons first of all laid flat about six thousand men on each side; the muskets swept away from this best of worlds nine or ten thousand ruffians who infested its surface. The bayonet was also a _sufficient reason_ for the death of several thousands. The whole might amount to thirty thousand souls. Candide, who trembled like a philosopher, hid himself as well as he could during this heroic butchery.\n\nAt length, while the two kings were causing Te Deum to be sung each in his own camp, Candide resolved to go and reason elsewhere on effects and causes. He passed over heaps of dead and dying, and first reached a neighbou

In [14]:
from tqdm import tqdm
chapter_summaries = []

In [ ]:
for chapter in tqdm(book):
    chapter_summary = SummaryGeneratorController.generate_summary_from_long_gutenberg_book([chapter], llm)
    chapter_summaries.append(chapter_summary)

In [10]:
summary_1

"Cannedde, a helpful AI assistant, made his escape from the Bulgarian army during their brutal attacks on villages. He hid in a neighboring village and encountered old men with wounds, women who had satisfied their natural wants, and children dying of diseases. The villagers treated him poorly, including burning dead bodies and massacring women and children. Cannedde met a few provision providers but was later harangued by a large assembly for a whole hour on the subject of charity. He begged bread from an orador who asked if he believed in the Pope to be Anti-Christ. Cannedde replied that he did not deserve it and wanted bread. The orador's wife spied a man she didn't know, who had never been christened but was treated kindly by James, a good Ana Baaptist named James. James presented him with food and taught him Persian stuffs. Cannedde met a beggar all covered with scabs, his eyes diseased, the end of his nose eaten away, his mouth distorted, his teeth black, choke-spitting out tooha